In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

from transformers import TFBertModel, BertTokenizer, TFBertMainLayer, BertConfig

In [ ]:
train_queries = pd.read_csv('../data/train_queries.csv')

In [ ]:
train_queries.info()

In [ ]:
fixed_index = pd.read_csv('../data/fixed_test_set_index_top100.csv', delimiter=',')

In [ ]:
fixed_index.info()

In [ ]:
relevance_judgments = pd.read_csv('../data/data_search_2_e_train_qrels.txt', delimiter=" ")
relevance_judgments = pd.DataFrame(relevance_judgments.values, columns = ["query_id", "dataset_id", "relevance"])

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU not found')
print('found GPU at {}'.format(device_name))

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

In [ ]:
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
bert_model = TFBertModel.from_pretrained("bert-base-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
MAX_TOKENS = 200

In [ ]:
ranking_model = tf.keras.models.load_model("../../news_table_matching/learning_to_rank_models/bert_based_models/bert_full_model_plus_attention_TM_01_0.9845.h5")

In [ ]:
embedding_model = gs.models.FastText.load_fasttext_format('../../news_table_matching/pre_trained_models/cc.en.300.bin')

In [ ]:
MAX_PAD = 55

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD:
        
        embedding = embedding_model.wv[value]
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD]]
        
        return embedding

In [ ]:
def get_relevance(query_id, dataset_id):
    
    relevance_by_query = relevance_judgments.loc[relevance_judgments['query_id'] == query_id]
    
    relevance_by_dataset = relevance_by_query.loc[relevance_by_query['dataset_id'] == dataset_id]
    
    if relevance_by_dataset.empty:
    
        return 0
    
    else:
    
        relevance_jugdment = relevance_by_dataset['relevance'].values[0].replace("L","")
        
        return int(relevance_jugdment)

In [ ]:
def dcg_metric(relevance,k):
    
    dcg = 0
    
    for i in range(1,k+1):
        
        dcg = dcg + (relevance[i-1] / np.log2(i+1))
        
    return dcg

In [ ]:
result_ndcg = []

for i, row in tqdm(train_queries.iterrows()):

    #current article values
    query_id = row['query_id']
    query_description = str(row['query_description'])
   
    #embedding query
    query_dataset_ids = []
    query_dataset_mask = []
    query_dataset_seg = []
    
    query_title = []
    dataset_title = []
    dataset_description = []
    

    #return index
    return_index = fixed_index.loc[fixed_index['label_index'] == query_id]
    

    dataset_index = []
    #creating embedding 
    for i, row in return_index.iterrows():
           
        dataset_title_text = str(row['dataset_title'])
        dataset_description_text = str(row['dataset_description'])

        #getting the bert embedding
        return_tokenizer2 = bert_tokenizer.encode_plus(
          query_description,
          dataset_title_text+" "+dataset_description_text,
          max_length=MAX_TOKENS,
          add_special_tokens=True,
          return_token_type_ids=True,
          pad_to_max_length=True,
          return_attention_mask=True,
        )

        #bert
        query_dataset_ids.append(return_tokenizer2['input_ids'])
        query_dataset_mask.append(return_tokenizer2['attention_mask'])
        query_dataset_seg.append(return_tokenizer2['token_type_ids'])
        
        #fasttext embedding
        query_title_embedding = create_embedding(query_description)
        dataset_title_embedding = create_embedding(dataset_title_text)
        dataset_description_embedding = create_embedding(dataset_description_text)

        query_title.append(query_title_embedding)
        dataset_title.append(dataset_title_embedding)
        dataset_description.append(dataset_description_embedding)

    
    #bert
    query_dataset_ids = np.array(query_dataset_ids)
    query_dataset_mask = np.array(query_dataset_mask)
    query_dataset_seg = np.array(query_dataset_seg)
    
    #fasttext
    query_title = np.array(query_title)
    dataset_title = np.array(dataset_title)
    dataset_description = np.array(dataset_description)

    ranking_model_by_dataset = ranking_model.predict([query_dataset_ids,query_dataset_mask,query_dataset_seg,query_title,query_title,dataset_title,dataset_description]) 
    
    ranked_datasets_model = []
    
    for i in range(0,len(ranking_model_by_dataset)):

        ranked_datasets_model.append([return_index.iloc[i]['dataset_id'],return_index.iloc[i]['dataset_title'],ranking_model_by_dataset[i][0]]) 

    data_frame = pd.DataFrame(ranked_datasets_model, columns = ['dataset_id', 'dataset_title','dataset_ranking']) 
    data_frame_sorting = data_frame.sort_values('dataset_ranking', ascending=False)
    selected_top = data_frame_sorting.head(20)
    
    #getting true relevance
    
    true_relevance = []
    
    relevance_by_query = relevance_judgments.loc[relevance_judgments['query_id'] == query_id]
    #sorting and reading the top10
    ideal_relevance = relevance_by_query.sort_values(by=['relevance'], ascending=False).head(20)
    
    for i, row in ideal_relevance.iterrows():
        
        dataset_id = row['dataset_id']
        value_relevance = int(row['relevance'].replace("L",""))
        
        true_relevance.append(value_relevance)

    #getting the predicted relevance
    predict_relevance = []
    
    for i,row in selected_top.iterrows():
        
        value_relevance = get_relevance(query_id,row['dataset_id'])
    
        predict_relevance.append(value_relevance)
    
    #computing NDCG
    dcg = dcg_metric(predict_relevance, len(predict_relevance))
    idcg = dcg_metric(true_relevance, len(true_relevance))
    
    if (idcg != 0):
    
        ndcg = dcg / idcg
    
    else:
        
        count = count + 1
        
        ndcg = 0
    
    result_ndcg.append(ndcg)

In [ ]:
np.mean(result_ndcg)